# Basic

## Setup

In [1]:
#| default_exp basic

In [3]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *

from fh_commons.core import *
from fh_commons.static import *

import json
import pandas as pd
from math import ceil

In [4]:
hdrs = bootstrap_hdrs()+download_js()+datatable_hdrs()

In [5]:
show(*hdrs)

In [6]:
#| eval: False
from ngrok_token import *
url = start_ngrok(token)

ngrok tunnel opened at: https://a223-3-238-95-91.ngrok-free.app


In [7]:
app,rt = fast_app(pico=False,hdrs=hdrs)
server = JupyUvi(app)

## Request

### req.url

Get current page's url

In [15]:
@rt
def sdf(req):
    return P(req.url)

In [18]:
htmx(url,'/sdf')

### req.url_for

Get original 

In [19]:
@rt
def sdf(req):
    return P(req.url_for('sdf'))

In [20]:
htmx(url,'/sdf')

With param:

In [35]:
# @rt("/api/{data}")
# def this(req,data:str):
#     return req.url_for('this',data=data)

In [36]:
# htmx(url,'/api/ssss')

## Toasts

toasts options: "info", "success", "warning", "error"

In [21]:
setup_toasts(app)

In [22]:
@rt
def test():
    return A('click to trigger toasts',hx_get=tt,target_id='content'),Div(id='content')

@rt
def tt(sess):
    return add_toast(sess, "Please upload a csv or excel file", "error")

In [29]:
htmx(url,'/test')

## Download file through a link

In [30]:
@app.get
def download_example():
    return FileResponse('data/example.csv')

@rt
def test(req):
    return A('Download Example File', 
             href=req.url_for('download_example'), # for dynamic route
             download='example.csv', # download file name
            )

In [31]:
htmx(url,'/test')

## Tooltip

In [7]:
#| export
def Tooltip(tooltip_content, # FT or str
            trigger_content, 
            element="span", 
            **kwargs):
    "Create a Bootstrap tooltip"
    
    # Convert tooltip content using to_xml() if it's an FT element, otherwise use directly
    tooltip_content = tooltip_content if isinstance(tooltip_content, str) else to_xml(tooltip_content)
    
    return ft(element,
                trigger_content,
                data_bs_toggle='tooltip', 
                data_bs_placement='right', 
              data_bs_custom_class='custom-tooltip', 
              data_bs_title=tooltip_content,
              data_bs_html='true',# Enable HTML content in tooltip
                **kwargs)

In [8]:
@rt
def test():
    return blank(),Tooltip(P('instruction'),'❔')

In [9]:
htmx(url,'/test')

In [10]:
#| export
def Helptip(tooltip_text=" ", # FT or str
            trigger_content=I(cls="bi bi-question-circle-fill") , 
            **kwargs):
    "Create a Bootstrap tooltip with help icon as default"

    return Tooltip(tooltip_text,trigger_content,style="margin-left: 4px;",)

In [11]:
@rt
def test():
    return blank(),P('Z Score',Helptip((P('sdfsdf'))))

In [12]:
htmx(url,'/test')

## Table / Dataframe

In [13]:
df = pd.read_csv('data/example.csv')

In [14]:
#| export
def df2html(df, 
             tooltips=None, # dict
             cls='table table-striped', 
             id=None, 
             **kwargs):
    """Create an HTML table with Bootstrap styling and tooltips"""
    # First create regular HTML table
    html = df.to_html(
        index=False,
        border=0,
        classes=cls,
        justify='left',
        table_id=id,
        escape=True,
        **kwargs)

    if tooltips:
        # Replace the header cells with tooltip versions
        for col, tooltip in tooltips.items():
            # Find the header cell containing this column name
            pattern = f'<th>({col})</th>'
            
            # Create the new header with tooltip - fixed HTML structure
            # \\1 preserves the original col name
            new_header = f'''<th>\\1 {to_xml(Helptip(tooltip))}</th>'''
            
            # Replace in HTML
            html = re.sub(pattern, new_header, html)

    return NotStr(html)

In [15]:
#| export
def download_table(*args, **kwargs):
    "Enable table download through a download button"
    return download_button(*args, onclick='downloadTableAsCSV(this)',**kwargs)

`download_table()`needs to be wrapped in the same Div with the `df2html`

In [16]:
@rt
def test():
    # to show table correctly, always needs a container
    return Container(
        Div(download_table(), df2html(df,tooltips={'ID': P('ssss')}))
    )

In [17]:
htmx(url,'/test')

### Download invisible dataframe

In [18]:
#| export
def send_df(df,fname, href, button=download_button(' Data')):
    json_data = df.to_json(orient='records')
    return Form(
        Hidden(value=json_data,id='dataframe'),
        Hidden(value=fname,id='fname'),
        button,
        method='post',
        action=href,
        style='display:inline;')

In [19]:
#| export
def download_df(dataframe,fname):
    data_dict = json.loads(dataframe)
    df = pd.DataFrame(data_dict)
    csv_string = df.to_csv(index=False)

    headers = {
        'Content-Disposition': f'attachment; filename="{fname}"'
    }
    return Response(csv_string, media_type='text/csv', headers=headers)

In [27]:
@app.post
def download_file(dataframe:str, fname:str):
    return download_df(dataframe,fname)

@rt
def test(req):
    download_button = send_df(df,
                            fname='sdf.csv',
                            href=req.url_for('download_file') # function name
                           )
    return download_button

In [28]:
htmx(url, '/test')

### Dynamic table

In [22]:
#| export
def dynamic_table(df,id='dynamic_table', cls='table table-striped',):
    "Dynamic data tables; make new id names if multiple tables"
    script = Script(f"new DataTable('#{id}')")
    return df2html(df,cls=cls,id=id),script

In [23]:
import numpy as np

In [24]:
matrix = np.random.rand(20, 10)
df = pd.DataFrame(matrix).round(2)

In [25]:
@rt
def test():
    return Container(dynamic_table(df))

In [26]:
htmx(url,'/test')

## Pagination

In [37]:
#| export
def get_pagination(page,total_pages,routes):
    "Bootstrap-styled pagination with integrated page jump form"
    return Nav(
        Ul(
            Li(A("Previous", href=f"{routes}?page={page-1}" if page > 1 else "#", 
                 cls="page-link" + (" disabled" if page <= 1 else "")), 
               cls="page-item"),
            Li(Span(f"Page {page} of {total_pages}", cls="page-link"), cls="page-item active"),
            Li(A("Next", href=f"{routes}?page={page+1}" if page < total_pages else "#", 
                 cls="page-link" + (" disabled" if page >= total_pages else "")), 
               cls="page-item"),
            Li(
                Form(
                    Input(type="number", name="page", placeholder="Page", min="1", max=str(total_pages), 
                          cls="form-control", style="width: 100px; display: inline-block;"),
                    Button("Go", type="submit", cls="btn btn-primary"),
                    cls="d-flex align-items-center m-2 g-2",
                    method="get",
                    action=routes
                ),
                cls="page-item"
            ),
            cls="pagination justify-content-center align-items-center"
        ),
        aria_label="Page navigation"
    )

Suppose I want a pagination bar for page 3 of 20

In [38]:
%%s
get_pagination(3,20,'#')

The above pagination can be combined with this function:

In [45]:
#| export
def get_page_index(current_page, total_items, items_per_page=12):
    "Get start index and end index given the current page number"
    
    total_pages = ceil(total_items / items_per_page) #ceil returns integer number, ceil(4.2) --> 5
    
    page = max(1, min(current_page, total_pages))  # Ensure page is within valid range
    
    start_index = (current_page - 1) * items_per_page
    end_index = start_index + items_per_page
    
    return start_index, end_index,total_pages

Suppose I have a list of 100 items, and I want each page have 12 items, so I need to calculate start index and end index given the current page (e.g., 2)

In [46]:
get_page_index(2,100,12)

(12, 24, 9)

Example for a pagination:

In [47]:
protein_list = [str(i) for i in range(100)]

In [50]:
@rt
def test(req, page: int = 1):

    # get item index
    start_idx, end_idx,total_pages = get_page_index(page, len(protein_list))

    # get items
    current_proteins = protein_list[start_idx:end_idx]
    show_proteins = P(','.join(current_proteins))

    # get dynamic route
    this_routes =req.url_for('test')

    # get pagination item
    pagination = get_pagination(page,total_pages,routes=this_routes)

    return show_proteins, pagination,this_routes

In [51]:
htmx(url,'/test')

## Modal

### Modal content

In [52]:
#| export
def modal_content(title, *args):
    return Div(
        Div(
            Div(
                H1(title, cls='modal-title fs-5'),
                Button(type='button', data_bs_dismiss='modal', aria_label='Close', cls='btn-close'),
                cls='modal-header'
            ),
            Div(*args, cls='modal-body'),
            cls='modal-content'
        ),
        cls='modal-dialog modal-xl'
    )

In [53]:
%%s
modal_content('Modal content','sdfsdf')

It can be combined with the below to search and trigger modal

### Use form button to trigger modal

In [54]:
#| export
def modal_form(*form_content,post, btn_text='Search',cls=None,target_id='modals-window'):
    "Reference: https://htmx.org/examples/modal-bootstrap/"
    modal_button = Form(*form_content, # Input(type='text',id='ssss')
                        Div(
                            Button(btn_text,cls='btn btn-primary py-3'),
                            data_bs_toggle='modal', 
                            data_bs_target=f'#{target_id}'),
                        post=post, 
                        target_id=target_id, 
                        hx_trigger='submit', 
                        cls=cls)

    modal_content =Div(
        Div(
            Div(cls='modal-content'),
            role='document',
            cls='modal-dialog modal-lg modal-dialog-centered'
        ),
        id=target_id,
        style='display: none',
        aria_hidden='false',
        tabindex='-1',
        cls='modal modal-blur fade'
    )
    return modal_button, modal_content

In [55]:
@rt
def page():
    modal_send = modal_form(Input(type='text',id='input_text'),post=modal)
    return modal_send

@rt
def modal(input_text:str):
    return modal_content('Modal title',input_text)

In [56]:
htmx(url,'/page')

### Use anchor link to trigger modal

In [57]:
#| export
def modal_link(text,post,cls=None,target_id='modals-window'):
    "Reference: https://htmx.org/examples/modal-bootstrap/"
    modal_button = A(text,
                     data_bs_toggle='modal', 
                     data_bs_target=f'#{target_id}',
                     post=post, 
                     target_id=target_id, 
                     cls=cls)

    modal_content =Div(
        Div(
            Div(cls='modal-content'),
            role='document',
            cls='modal-dialog modal-lg modal-dialog-centered'
        ),
        id=target_id,
        style='display: none',
        aria_hidden='false',
        tabindex='-1',
        cls='modal modal-blur fade'
    )
    return modal_button, modal_content

In [58]:
@rt
def page():
    modal_send = modal_link('click here',post=modal)
    return modal_send

@rt
def modal():
    return modal_content('Modal title','sdf')

In [59]:
htmx(url,'/page')

### Modal content

In [52]:
#| export
def modal_content(title, *args):
    return Div(
        Div(
            Div(
                H1(title, cls='modal-title fs-5'),
                Button(type='button', data_bs_dismiss='modal', aria_label='Close', cls='btn-close'),
                cls='modal-header'
            ),
            Div(*args, cls='modal-body'),
            cls='modal-content'
        ),
        cls='modal-dialog modal-xl'
    )

In [53]:
%%s
modal_content('Modal content','sdfsdf')

It can be combined with the below to search and trigger modal

### Use form button to trigger modal

In [54]:
#| export
def modal_form(*form_content,post, btn_text='Search',cls=None,target_id='modals-window'):
    "Reference: https://htmx.org/examples/modal-bootstrap/"
    modal_button = Form(*form_content, # Input(type='text',id='ssss')
                        Div(
                            Button(btn_text,cls='btn btn-primary py-3'),
                            data_bs_toggle='modal', 
                            data_bs_target=f'#{target_id}'),
                        post=post, 
                        target_id=target_id, 
                        hx_trigger='submit', 
                        cls=cls)

    modal_content =Div(
        Div(
            Div(cls='modal-content'),
            role='document',
            cls='modal-dialog modal-lg modal-dialog-centered'
        ),
        id=target_id,
        style='display: none',
        aria_hidden='false',
        tabindex='-1',
        cls='modal modal-blur fade'
    )
    return modal_button, modal_content

In [55]:
@rt
def page():
    modal_send = modal_form(Input(type='text',id='input_text'),post=modal)
    return modal_send

@rt
def modal(input_text:str):
    return modal_content('Modal title',input_text)

In [56]:
htmx(url,'/page')

### Use anchor link to trigger modal

In [57]:
#| export
def modal_link(text,post,cls=None,target_id='modals-window'):
    "Reference: https://htmx.org/examples/modal-bootstrap/"
    modal_button = A(text,
                     data_bs_toggle='modal', 
                     data_bs_target=f'#{target_id}',
                     post=post, 
                     target_id=target_id, 
                     cls=cls)

    modal_content =Div(
        Div(
            Div(cls='modal-content'),
            role='document',
            cls='modal-dialog modal-lg modal-dialog-centered'
        ),
        id=target_id,
        style='display: none',
        aria_hidden='false',
        tabindex='-1',
        cls='modal modal-blur fade'
    )
    return modal_button, modal_content

In [58]:
@rt
def page():
    modal_send = modal_link('click here',post=modal)
    return modal_send

@rt
def modal():
    return modal_content('Modal title','sdf')

In [59]:
htmx(url,'/page')

In [73]:
#| export
def get_spinner(id, cls='d-flex justify-content-center'):
    return Div(Div(
        Span('Loading...', cls='visually-hidden'),
        role='status',
        cls='htmx-indicator spinner-border',
        id=id,
    ),cls=cls)

In [75]:
%%s
get_spinner(id='sdf')

Example:

In [74]:
# @app.post
# def example():
#     ...
#     send_select = Form(
#             Hidden(value=df.to_json(),id='upload_df'),
#             Div(select_list,button,cls='row g-2 align-items-center'),
#             post=calculate,
#             target_id='result',
#             hx_indicator='#spinner',
#         )
    
#     spinner=get_spinner(id='spinner')
    
#     return send_select, spinner, ...


## End

In [29]:
#| hide
import nbdev; nbdev.nbdev_export()

In [78]:
server.stop()
kill_ngrok()

ngrok tunnel killed
